# 1. Creació de l'estructura on es guardaran les dades
Es defineixen dues classes:
<p>1) Classe municipi
<p>2) Classe partit
<p>La primera classe detalla tots els atributs que conté cada partit polític com vots, el percentatge de vots respecte el total, majoria absoluta, majria relativa i empats.
<p>La segona classe detalla tots els atributs que conté cada partit polític com vots, el percentatge de vots respecte el total i regidors.

In [1]:
class municipi:
    # Inicialitzador de la classe
    def __init__(self, nom, url):
        self.__nom = nom
        self.__url = url
        self.partits = []
        self.cens = 0
        self.escrutat = '00,00%'
        self.abstencio = 0
        self.vots_comp = 0
        self.vots_nuls = 0
        self.vots_blanc = 0
    # Getters de cadascun dels atributs
    def getNom(self):
        return self.__nom
    def getUrl(self):
        return self.__url
    def getPartit(self,index):
        return self.partits[index]
    def getLengthPartits(self):
        return len(self.partits)
    def setPartit(self,partit):
        self.partits.append(partit)
    def setAtributs(self,cens,escrutat,abstencio,vots_comp,vots_nuls,vots_blanc):
        self.cens = cens
        self.escrutat = escrutat
        self.abstencio = abstencio
        self.vots_comp = vots_comp
        self.vots_nuls = vots_nuls
        self.vots_blanc = vots_blanc 
    def getAtributs(self):
        return print('Nom: ',self.nom,' Cens: ',self.cens,' Escrutat: ',self.escrutat,\
                     'Abstencio: ',self.abstencio,'Vots comptabilitats: ',self.vots_comp,\
                     ' Vots nuls: ',self.vots_nuls,' Vots blancs: ',self.vots_blanc)
class partit:
    # Inicialitzador de la classe
    def __init__(self,nom,acronim,vots,percent,regidors):
        self.__nom = nom
        self.__acronim = acronim
        self.__vots = vots
        self.__percent = percent
        self.__regidors = regidors
    # Getters de cadascun dels atributs
    def getNom(self):
        return self.__nom
    def getAcronim(self):
        return self.__acronim
    def getVots(self):
        return self.__vots
    def getPercent(self):
        return self.__percent
    def getRegidors(self):
        return self.__regidors
    def getTot(self):
        return print('Nom: ',self.__acronim,' Vots: ',self.__vots,' %: ',self.__percent,\
                     ' Regidos: ',self.__regidors)

  

<p> A continuació es detallen les funcions que s'utilitzen per la manipulació i extraccio de les dades:
<p>1) llistaMunicipis: Funció que cerca el llistat de municipis de les eleccions a Catalunya.  

In [2]:
def llistaMunicipis(llista_municipis,llista_municipis_1,web):
    # INPUT: Directori web on cerca els municipis i llistes municipis buides
    # OUTPUT: Retorna llistes amb tots els municipis i la seva URL associades

    # Càrrega i obtenció de la pàgina web
    page = requests.get(web)
    soup = BeautifulSoup(page.content,"html.parser")
    
    counter = 1 #variable contador que indica a quina linia es treballa
    for tag in soup.find_all('li', class_ = 'list-group-item'):
        # Els municipis comencen al 47 i acaben al 993. (1-47 provincies i comarques de catalunya i del 994-final C.C.A.A).
        if counter >= 47 and counter <= 993: 
            # S'obté el nom de cada municipi i la seva URL
            nom = tag.a.string # S'accedeix al nom
            url =tag.a['href'] # S'accedeix a la url

            # S'afegueix el municipi a la llista de municipis
            llista_municipis.append(municipi(nom,url))
            llista_municipis_1.append(municipi(nom,url))
        #S'incrementa en 1 el contador
        counter +=1

<p> 2) lecturaMunicipis: Funció que extreu d'una taula HTML la informació relativa a cada partit polític corresponent a un municipi.

In [3]:
def lecturaMunicipis(tag_taula,municipi):
    # INPUT: tag_taula: tag amb la taula HTML amb els partits
    #        municipi: variable tipus municipi amb la llista de partits buida
    # OUTPUT: municipi modificada ambla llista de partits plena
    
    # llistes nom_partits i acronim partits buides
    noms_partits = []
    acronim_partits = []
    # Llista auxiliar valors buida
    valors = []
    
    # Extracció de nom i l'acrònim de cada partit
    for i in tag_taula.find_all('td',class_ ="linkeleccionsd3 atribpartido"):
        noms_partits.append(i['title'])
        acronim_partits.append(i.findNext(text = True))


    # Extracció dels valors {vots,%,Regidors}, per cada partit->3 valors.
    for i in tag_taula.find_all('td',class_ ="linkeleccionsd3 text-right"):
        valors.append(i.findNext(text = True))

    # Es junta tot
    i = 0
    j = 0

    if len(noms_partits)*3 == len(valors):
        while i<len(valors):
            vots = valors[i]
            i +=1
            percent = valors[i]
            i +=1
            regidors = valors[i]
            i +=1
            partt = partit(noms_partits[j],acronim_partits[j],vots,percent,regidors)
            municipi.setPartit(partt)
            #print('Nom: ',acronim_partits[j],' Vots: ',vots,' %: ',percent,' Regidos: ',regidors)
            j +=1
        #print("Obtenció dades correcte")
    else:
        print("¡¡ERROR!! - Obtenció dades incorrecte")

<p> 3) lecturaAtributs: Funció que extreu d'una taula HTML els atribut corresponents a cada localitat. La informació que s'extreu és Cens, total escrutat (%), abstencions, vots comptabilitzats, vots nuls i vots en blanc.

In [4]:
def lecturaAtributs(tag,eleccions):
    # INPUT: tag: variable tipus tag que conté la taula amb les dades de cens, etc, del municipi
    #        eleccions: llista de classe municipi   
    # OUTPUT: Variable municipi actualitzada amb els atributs
    
    aux = tag.findNext('td')
    cens = aux.findNext('td')
    aux = cens.findNext('td')
    aux = aux.findNext('td')
    escrutat = aux.findNext('td')
    aux = escrutat.findNext('td')

    vots_comp = aux.findNext('td')
    vots_comp_percent = vots_comp.findNext('td')

    aux = vots_comp_percent.findNext('td')
    abs_comp = aux.findNext('td')
    abs_comp_percent = abs_comp.findNext('td')

    aux = abs_comp_percent.findNext('td')
    vots_nuls = aux.findNext('td')
    vots_nuls_percent = vots_nuls.findNext('td')

    aux = vots_nuls_percent.findNext('td')
    vots_blanc = aux.findNext('td')
    vots_blanc_percent = vots_blanc.findNext('td')
    
    # Es mostra per pantalla les dades extretes de cada municipi com a check point
    eleccions.setAtributs(cens.string,escrutat.string,abs_comp.string,vots_comp.string,\
                          vots_nuls.string,vots_blanc.string)

4) extraccioDemarcacio: Funció 

In [5]:
def extraccioDemarcacio(web):
    # INPUT:
    # OUTPUT:
    
    df_demarcacio = pd.DataFrame(columns=['id_Municipi', 'municipi','comarca', 'provincia'])
    page = requests.get(web)
    soup = BeautifulSoup(page.content,"html.parser")
    a = soup.find(id = 'selectdemarcacio') #Selector Demarcació
    b = soup.find(id = 'selectcomarca') #Selector Comarca
    c = soup.find(id = 'selectmunicipi') #Selector Municipi
    
    counter = 0
    for i in c.find_all('option'):
        if counter != 0:
            i.string #Nom
            i['data-comarca'] #id comarca
            i['data-demarcacio'] #id demarcació
            i['value'] # id nom
            df_demarcacio = df_demarcacio.append({'id_Municipi': int(i['value']), 'municipi': i.string,\
                                                  'comarca':i['data-comarca'], 'provincia':i['data-demarcacio']},\
                                                 ignore_index=True)        
        counter += 1

    counter = 0
    for i in b.find_all('option'):
        i.string #comarca
        i['value']# id_comarca
        df_demarcacio['comarca'].replace({i['value']: i.string}, inplace=True)

    counter = 0
    for i in a.find_all('option'):
        i.string #provincia
        i['value']# id_provincia
        df_demarcacio['provincia'].replace({i['value']: i.string}, inplace=True)
   
    return df_demarcacio

<p> 5) extraccioDades: Funció que llegeix la llista de municipis i extreu tota la informació de les eleccions de 2019 i 2015 per municipis.

In [6]:
def extraccioDades(elec_2019,elec_2015):
    # INPUT: elec_20XX: llelec_2019ista de classe municipi amb noms i urls de cadascun ()
    # OUTPUT: elec_20XX: modificada amb els atributs de cada municipi i la llsta de partits polítics
    
    counter = 0 # S'inicia contador a 0
    while counter < len(elec_2019):
    #while counter < 12: #Test de 1 municipis
        # Es fa la lectura de URL de cada municipi
        url = elec_2019[counter].getUrl()
        # Càrrega i obtenció de la pàgina web
        page = requests.get(url)
        while page.status_code != 200:
            print ("Error de connexió del típus {}".format(page.status_code))
            time.sleep(5)
            page = requests.get(url)
        
        #page = ''
        #while page == '':
        #    try:
        #        page = requests.get(url)
        #        break
        #    except:
        #        print("Connection refused by the server..")
        #        time.sleep(5)
        #        continue
        
        soup = BeautifulSoup(page.content,"html.parser")
        
        # Obtenció de les taules en format TAG
        taula_2019 = soup.find('table')             # Partits polítics 2019
        dades_2019 = taula_2019.find_next('table')  # Informació extra 2019
        taula_2015 = dades_2019.find_next('table')  # Partits polítics 2015
        dades_2015 = taula_2015.find_next('table')  # Informació extra 2015
              
        # S'extreuen els atributs dels municipis
        lecturaAtributs(dades_2019,elec_2019[counter])
        #elec_2019[counter].getAtributs()
        lecturaAtributs(dades_2015,elec_2015[counter])
        #elec_2015[counter].getAtributs()
        #S'extreuen els partits politics de cada municipi
        lecturaMunicipis(taula_2019,elec_2019[counter])
        lecturaMunicipis(taula_2015,elec_2015[counter])
        
        print(counter)
               
        counter += 1


# 2. Descarrega de les dades
Es descarrega la informació del lloc web d'interes, en aquest cas de les eleccions del 2015 i 2019 per municipis a catalunya.


In [7]:
# Carrega de llibreries necessaries
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup

In [8]:
# Es defineix la web de recerca
web = "https://www.naciodigital.cat/municipals2019"

# Es defineix les llistes de municipis buides. Els municipis són els mateixos per 2015 i 2019.
elec_2019 = []
elec_2015 = []

# Es crida la funció llistaMunicipis()
llistaMunicipis(elec_2019,elec_2015,web)

# S'extreuen les dades
extraccioDades(elec_2019,elec_2015)
#elec_2019[0].partits[0].getTot()
#elec_2015[0].partits[0].getTot()

# Extracció dades geogràfiques {provincia,comarca,municipi}
# És indiferent la url utilitzada, totes contenen la mateixa informació geogràfica
web1 = ("https://www.naciodigital.cat/municipals2019/municipi/25001/abella/conca") 
# Es crida la funció extraccioDemarcacio
df_demarcacio = extraccioDemarcacio(web1)
print("Dataframe Demarcació: ", df_demarcacio, sep='\n')
#a = df_demarcacio.loc[df_demarcacio['municipi'] == 'Abrera']
#print(a)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# 3. Preparació de les dades
<p>Es começa a preparar el set de dades per poder-ho exportar. Es preparen dos dataset que contenen tota la informació.<p> Es preparen  datasets:
<p>1) ResultatsGlobals
<p>2) ResultatsCand

In [9]:
# df_RG correspon al dataframe generat per incloure tota la informació del dataset ResultatsGlobals
# Es defineix el dataframe df_RG
df_RG =  pd.DataFrame(columns=['id_Municipi', 'municipi','comarca', 'provincia','url','cens_2019',\
                                              'escrutat_2019','abstencio_2019','comptabilitzat_2019','null_2019','blanc_2019',\
                                             'cens_2015','escrutat_2015','abstencio_2015','comptabilitzat_2015','null_2015',\
                                              'blanc_2015'])
df_RC = pd.DataFrame(columns=['id_Candidatura','nom_Candidatura','acronim_Candidatura','id_Municipi','nom_Municipi','vots'\
                              ,'regidors','any'])


In [10]:
# Sobté ResultatsGlobals
counter = 0
while counter < len(elec_2019):
#while counter < 12:
    # Es troba la coincideincia del dataframe df_demarcacio amb la llista de municipis del any 2019
    aux = df_demarcacio.loc[df_demarcacio['municipi'] == elec_2019[counter].getNom()]
    # Es carreguen el valors individualment
    df_RG = df_RG.append({'id_Municipi':aux['id_Municipi'].values[0], 'municipi':aux['municipi'].values[0],\
                          'comarca':aux['comarca'].values[0],'provincia':aux['provincia'].values[0],\
                          'url':elec_2019[counter].getUrl(),'cens_2019':elec_2019[counter].cens,\
                          'escrutat_2019':elec_2019[counter].escrutat,'abstencio_2019':elec_2019[counter].abstencio,\
                          'comptabilitzat_2019':elec_2019[counter].vots_comp,'null_2019':elec_2019[counter].vots_nuls,\
                          'blanc_2019':elec_2019[counter].vots_blanc,'cens_2015':elec_2015[counter].cens,\
                          'escrutat_2015':elec_2015[counter].escrutat,'abstencio_2015':elec_2015[counter].abstencio,\
                          'comptabilitzat_2015':elec_2015[counter].vots_comp,'null_2015':elec_2015[counter].vots_nuls,\
                          'blanc_2015':elec_2015[counter].vots_blanc},ignore_index=True)  
    
    counter += 1
# Es mostren els primers 5 municipis
df_RG.head(5)


,id_Municipi,municipi,comarca,provincia,url,cens_2019,escrutat_2019,abstencio_2019,comptabilitzat_2019,null_2019,blanc_2019,cens_2015,escrutat_2015,abstencio_2015,comptabilitzat_2015,null_2015,blanc_2015
0,25001,Abella de la Conca,Pallars Jussà,Lleida,https://www.naciodigital.cat/municipals2019/mu...,146,"100,00%",24,122,0,0,140,"100,00%",32,108,3,0
1,8001,Abrera,Baix Llobregat,Barcelona,https://www.naciodigital.cat/municipals2019/mu...,9.100,"100,00%",3.240,5.860,16,32,8.851,"100,00%",3.628,5.223,55,60
2,25003,Agramunt,Urgell,Lleida,https://www.naciodigital.cat/municipals2019/mu...,3.774,"100,00%",1.279,2.495,50,119,3.831,"100,00%",1.453,2.378,37,56
3,8002,Aguilar de Segarra,Bages,Barcelona,https://www.naciodigital.cat/municipals2019/mu...,221,"100,00%",30,191,5,3,203,"100,00%",58,145,10,20
4,17001,Agullana,Alt Empordà,Girona,https://www.naciodigital.cat/municipals2019/mu...,600,"100,00%",130,470,3,12,611,"100,00%",143,468,8,19


In [11]:
# Sobté ResultatsCand
counter = 0
id_Candidatura = 0
while counter < len(elec_2019):
#while counter < 1:
    # Es troba la coincideincia del dataframe df_demarcacio amb la llista de municipis del any 2019
    aux = df_demarcacio.loc[df_demarcacio['municipi'] == elec_2019[counter].getNom()]
    i = 0
    while i < len(elec_2019[counter].partits):
        # Es carreguen el valors individualment
        df_RC = df_RC.append({'id_Candidatura':id_Candidatura,'nom_Candidatura':elec_2019[counter].partits[i].getNom(),\
                              'acronim_Candidatura':elec_2019[counter].partits[i].getAcronim(),\
                              'id_Municipi':aux['id_Municipi'].values[0],\
                              'nom_Municipi':elec_2019[counter].getNom(),'vots':elec_2019[counter].partits[i].getVots()\
                              ,'regidors':elec_2019[counter].partits[i].getRegidors(),'any':int(2019)},ignore_index=True) 
        i += 1
        id_Candidatura += 1  
        
    # Es troba la coincideincia del dataframe df_demarcacio amb la llista de municipis del any 2015
    aux = df_demarcacio.loc[df_demarcacio['municipi'] == elec_2015[counter].getNom()]
    i = 0
    while i < len(elec_2015[counter].partits):
        # Es carreguen el valors individualment
        df_RC = df_RC.append({'id_Candidatura':id_Candidatura,'nom_Candidatura':elec_2015[counter].partits[i].getNom(),\
                              'acronim_Candidatura':elec_2015[counter].partits[i].getAcronim(),\
                              'id_Municipi':aux['id_Municipi'].values[0],\
                              'nom_Municipi':elec_2015[counter].getNom(),'vots':elec_2015[counter].partits[i].getVots()\
                              ,'regidors':elec_2015[counter].partits[i].getRegidors(),'any':int(2015)},ignore_index=True) 
        i += 1
        id_Candidatura += 1             

    
    counter += 1
# Es mostren els primers 5 municipis
df_RC.head(20)

,id_Candidatura,nom_Candidatura,acronim_Candidatura,id_Municipi,nom_Municipi,vots,regidors,any
0,0,ESQUERRA REPUBLICANA DE CATALUNYA-ACORD MUNICIPAL,ERC-AM,25001,Abella de la Conca,64,4,2019
1,1,COMPROMÍS PEL MUNICIPI-CANDIDATURA DE PROGRÉS,CM-CP,25001,Abella de la Conca,43,1,2019
2,2,JUNTS PER ABELLA DE LA CONCA,JUNTS,25001,Abella de la Conca,15,0,2019
3,3,JOVES INDEPENDENTS PER ABELLA I BOIXOLS - ACOR...,JIAB - AM,25001,Abella de la Conca,53,4,2015
4,4,INDEPENDENTS PER ABELLA DE LA CONCA - CANDIDATURA,IxAC-CP,25001,Abella de la Conca,34,1,2015
5,5,CONVERGÈNCIA I UNIÓ,CiU,25001,Abella de la Conca,18,0,2015
6,6,PARTIT DELS SOCIALISTES DE CATALUNYA - CANDIDA...,PSC-CP,8001,Abrera,3.006,11,2019
7,7,ESQUERRA REPUBLICANA DE CATALUNYA - Acord Muni...,ERC - AM,8001,Abrera,810,2,2019
8,8,ALTERNATIVA D'ABRERA,Ad'Ab,8001,Abrera,653,2,2019
9,9,ABRERA EN COMÚ-EN COMÚ GUANYEM,ABRERA EN COMÚ-ECG,8001,Abrera,533,1,2019


<p> Es reaitzen les exportacions de dataframes a .csv:

In [12]:
df_RG.to_csv(r'ResultatsGlobals.csv', index = False, header=True)
df_RC.to_csv(r'ResultatsCand.csv', index = False, header=True)

#Error 
SSLError: HTTPSConnectionPool(host='www.naciodigital.cat', port=443): Max retries exceeded with url: /municipals2019/municipi/08127/monistrol/montserrat (Caused by SSLError(SSLError("bad handshake: SysCallError(10054, 'WSAECONNRESET')")))